In [ ]:
import tensorflow as tf
import os

import numpy as np

class Vgg19(object):
    def __init__(self, x, num_classes, keep_pro, weights_path = None, skip)
    self.x = x
    self.num_classes = num_classes
    self.keep_pro = keep_pro
    
    if weights_path is not None:
        self.weights_path = weights_path
        self.weights_dict = np.load(self.weights_path, encoding = "latin1").item()
    else:
        self.weights_path = None
    # skip layers which will load pretrianed weights
    # leave some layer which is not trainable
    self.skip = skip
    self.buildVgg()
    
    # train_mode 
    def buildVgg(self, train_mode = True):
        conv1_1 = self.conv2d("conv1_1", self.x, kernel = 3, stride = 1, out_dimension = 64)
        conv1_2 = self.conv2d("conv1_2", conv1_1, kernel = 3, stride = 1, out_dimension = 64)
        pool1 = self.pool2d("pool1", conv1_2, ksize = 2, stride = 2)
        
        conv2_1 = self.conv2d("conv2_1", pool1, kernel = 3, stride = 1, out_dimension = 128)
        conv2_2 = self.conv2d("conv2_2", conv2_1, kernel = 3, stride = 1, out_dimension = 128)
        pool2 = self.pool2d("pool2", conv2_2, ksize = 2, stride = 2)
        
        conv3_1 = self.conv2d("conv3_1", pool2, kernel = 3, stride = 1, out_dimension = 256)
        conv3_2 = self.conv2d("conv3_2", conv3_1, kernel = 3, stride = 1, out_dimension = 256)
        conv3_3 = self.conv2d("conv3_3", conv3_2, kernel = 3, stride = 1, out_dimension = 256)
        conv3_4 = self.conv2d("conv3_4", conv3_3, kernel = 3, stride = 1, out_dimension = 256)
        pool3 = self.pool2d("pool3", conv3_4, ksize = 2, stride = 2)
        
        conv4_1 = self.conv2d("conv4_1", pool3, kernel = 3, stride = 1, out_dimension = 512)
        conv4_2 = self.conv2d("conv4_2", conv4_1, kernel = 3, stride = 1, out_dimension = 512)
        conv4_3 = self.conv2d("conv4_3", conv4_2, kernel = 3, stride = 1, out_dimension = 512)
        conv4_3 = self.conv2d("conv4_4", conv4_3, kernel = 3, stride = 1, out_dimension = 512)
        pool4 = self.pool2d("pool4", conv4_4, ksize = 2, stride = 2)
        
        conv5_1 = self.conv2d("conv5_1", pool4, kernel = 3, stride = 1, out_dimension = 512)
        conv5_2 = self.conv2d("conv5_2", conv5_1, kernel = 3, stride = 1, out_dimension = 512)
        conv5_3 = self.conv2d("conv5_3", conv5_2, kernel = 3, stride = 1, out_dimension = 512)
        conv5_4 = self.conv2d("conv5_4", conv5_3, kernel = 3, stride = 1, out_dimension = 512)
        pool5 = self.pool2d("pool5", conv5_4, ksize = 2, stride = 2)
        
        fc_in = tf.reshape(pool5, shape = [-1, 7*7*512])
        fc6 = self.fcLayer("fc6", fc_in, 7*7*512, 4096, reluFlag = True)
        # if train_mode is True, then the model will apply dropout,
        # for test, the dropout = 1.0, which means no dropout
        dropout6 = tf.cond(train_mode, lambda: tf.nn.dropout(fc6, self.keep_pro), lambda: fc6)
        
        fc7 = self.fcLayer("fc7", dropout6, 4096, 4096, reluFlag = True)
        # same as fc6
        dropout7 = tf.cond(train_mode, lambda: tf.nn.dropout(fc7, self.keep_pro), lambda: fc7)
        # out_size = num_classes
        fc8 = self.fcLayer("fc8", dropout7, 4096, self.num_classes, reluFlag = True)
        
        self.out = tf.nn.softmax(fc8, name = "out")
        
        
    def _get_var(self, name, idx, var_name, shape, initializer):
        """
        args:
            name: scope name
            idx: one scope include "weights" and "biases", idx is index of variables
            var_name: "weights" and "biases"
            initial_value: to initialize the variables
        """
        if self.weights_path is not None and name in self.weights_dict:
            if name not in self.skip:
                with tf.variable_scope(name, reuse = True):
                    value = sess.run(tf.get_variable(var_name, trainable = False).assign(self.weights_dict[name][idx]))
            value = sess.run(tf.get_variable(var_name, trainable = True).assign(self.weights_dict[name][idx]))
        else:
            value = tf.get_variable(name = var_name, shape = shape, initializer = initializer, dtype = tf.float32)
               
        return value
    
    def conv2d(self, name, input_x, kernel, stride, out_dimension, padding = "SAME"):
        """
        args:
            name: scope_name, to check if the scope_name is in weights_dict, then get pretrainer weights
            input_x: input for convolutional layer, shape = [batch_size, height, width, dimension]
            kernel: kernel size
            stride: stride for convolution caculation
            out_dimension: the dimension for output
        """
        witf tf.scope_name(name) as scope:
            in_dimension = int(input_x.get_size().as_list()[-1])
            w = self._get_var(name, 0, "weights", shape = [kernel, kernel, in_dimension, out_dimension],
                             initializer = tf.truncated_normal(stddev = 0.01))
            b = self._get_var(name, 1, "biases", shape = [out_dimension],
                             initializer = tf.constant_initializer(0.0))
            out = tf.nn.conv2d(input_x, w, stride = [1, stride, stride, 1], padding = padding)
            out = tf.bias_add(out, b)
            conv = tf.relu(out)
        return conv
    
    def pool2d(self, name, input_x, ksize, stride, padding = "SAME"):
        """
        args:
            name: name for operation
            input_x: input for pooling layer
            ksize: the size of window, shape = [1, height, width, 1]
            stride: the stride step shape = [1, height, width, 1]
        """
        return tf.nn.max_pool(input_x, ksize = [1, ksize, ksize, 1], stride = [1, stride, stride, 1]
                                 padding = padding, name = name)
    
    def fcLayer(self, name, input_x, in_size, out_size, reluFlag):
        """
        args:
            name: the name for scope
            input_x: input for fully connected layer
            in_size, out_size: size to determine the kernel in fc layer
            reluFlag: flag determines if the out will use relu
        """
        with tf.scope_name(name) as scope:
            w = self._get_var(name, 0, "weights", shape = [in_size, out_size],
                              initializer = tf.truncated_normal(stddev = 0.01))
            b = self._get_var(name, 1, "biases", shape = [out_size],
                             initializer = tf.constant_initializer(0.0))
            out = tf.nn.bias_add(tf.matmul(input_x, w), b)
            if reluFlag:
                fc = tf.nn.relu(out)
            else:
                fc = out
        return fc
    
